# Linearization, stability, observability, and controllability for Model 1 and 2.

In [8]:
# Author: Bernt Lie
# Edited on purpose and use by: Madhusudhan Pandey
# Necessary packages
using DifferentialEquations
using LinearAlgebra
using Plots
using Plots.PlotMeasures
using LaTeXStrings
pyplot();

In [9]:
# ODE model of Model1
function model1(x,u,t)
    # unpacking states
    Tr = x[1]
    Ts = x[2]
    TFe = x[3]
    # upacking inputs
    Twc = u[1]
    Ifd = u[2]
    It = u[3]
    QdFes = u[4]
    Wdf = u[5]
    mdw = u[6]
    mda = u[7]
    # parameters
    chpa = 1.15
    chpw = 4.2
    chpCu = 0.385
    chpFe = 0.465
    # Masses
    mr = 9260.
    ms = 6827.
    mFe = 71200.
    # Heat transfer coefficients
    UAr2d = 2.7
    UAs2Fe = 20.
    UAFe2a = 14.3
    hAax = 55.6
    hAwx = 222.
    UAx = 1/(1/hAax+1/hAwx)
    # Resistances
    Rr = 0.127e-3
    Rs = 1.95e-6
    alphaCu=0.00404
    TCu_o=25.
    #
    Qdfs = 0.8*Wdf
    # Stanton numbers
    NSta = UAx/chpa/mda
    NStw = UAx/chpw/mdw
    NStd = NStw - NSta
    # Matrices
    M1 = diagm(0=>[mr*chpCu , ms*chpCu , mFe*chpFe])
    M2 = [-UAr2d 0. 0.; 0. -UAs2Fe UAs2Fe; 0. UAs2Fe -UAs2Fe-UAFe2a]
    M3 = [0. UAr2d 0.; 0. 0. 0.; 0. 0. UAFe2a]
    #
    N1 = [-mda*chpa mda*chpa+UAr2d 0.; 0. -mda*chpa mda*chpa+UAFe2a; NStw-NSta*
       exp(-NStd) 0. -NStd]
    N2 = [UAr2d 0. 0.; 0. 0. UAFe2a; 0. 0. 0.]
    #
    v = [1.1*Rr*Ifd^2, 3*Rs*It^2, QdFes]
    w = [Qdfs, 0., NSta*(1-exp(-NStd))*Twc]
    #
    z = N1\(N2*x + w)
    dxdt = M1\(M2*x+M3*z + v)
    return dxdt
end

# Inputs
u = [3.8,1055.,5360.,212,528,53.9,49.2]
# Initial states
x0 = [28., 28., 28.]
# Time span
tspan = (0., 300*60.)

(0.0, 18000.0)

In [10]:
# findind steady state solution
prob_steady_state = ODEProblem(model1,x0,(0.,Inf),u)
sol_steady_state = solve(prob_steady_state,DynamicSS(Tsit5()))
# steady state state values
xs = sol_steady_state.u[end,1]
# nominal inputs
us = u

┌ Warning: Instability detected. Aborting
└ @ DiffEqBase C:\Users\pande\.julia\packages\DiffEqBase\3gIPB\src\integrator_interface.jl:162


7-element Array{Float64,1}:
    3.8
 1055.0
 5360.0
  212.0
  528.0
   53.9
   49.2

In [11]:
using ForwardDiff
# functions in states and inputs
fx(x) = model1(x,us,0)
fu(u) = model1(xs,u,0)
# Jacobians at operating points
A = ForwardDiff.jacobian(fx,xs)
B = ForwardDiff.jacobian(fu,us)

3×7 Array{Float64,2}:
 0.000591328  8.26813e-5  0.0         0.0         …  -1.87037e-5  -2.55588e-5
 0.0          0.0         2.38594e-5  0.0             0.0          0.0       
 0.000269202  0.0         0.0         3.02042e-5     -8.51485e-6  -5.87091e-5

In [12]:
# Calculating time constants
# Eigen value of system matrix, A
using LinearAlgebra
# Eigenvector
R = eigvecs(A)
# Eigenvalues
λ = eigvals(A)
# Time Constants
τ = - @. inv(λ)/60

3-element Array{Float64,1}:
 23.521373643877883 
 62.693739754322685 
  2.0237132526949364

In [13]:
# Controllability and observability
using ControlSystems

# Rank of matrix, A
# System order
rank_of_A = rank(A)
# Controllability matrix
ctrb_C = ctrb(A,B)
# Rank of controlability matrix
rank_of_ctrb_C=rank(ctrb_C)

# Observability
C = [0. 1. 1.]
# Observability matrix
obsv_O = obsv(A,C)
rank_of_obsv_O=rank(obsv_O)

3

In [14]:
# ODE model of Model2
function model2(x,u,t)
    # unpacking states
    Tr = x[1]
    Ts = x[2]
    TFe = x[3]
    # upacking inputs
    Twc = u[1]
    Ifd = u[2]
    It = u[3]
    QdFes = u[4]
    Wdf = u[5]
    mdw = u[6]
    mda = u[7]
    # parameters
    chpa = 1.15
    chpw = 4.2
    chpCu = 0.385
    chpFe = 0.465
    # Masses
    mr = 9260.
    ms = 6827.
    mFe = 71200.
    # Heat transfer coefficients
    UAr2d = 2.7
    UAs2Fe = 20.
    UAFe2a = 14.3
    hAax = 55.6
    hAwx = 222.
    UAx = 1/(1/hAax+1/hAwx)
    # Resistances
    Rr = 0.127e-3
    Rs = 1.95e-6
    alphaCu=0.00404
    TCu_o=25.
    #
    Qdfs = 0.8*Wdf
    # Stanton numbers
    NSta = UAx/chpa/mda
    NStw = UAx/chpw/mdw
    NStd = NStw - NSta
    # Matrices
    M1 = diagm(0=>[mr*chpCu , ms*chpCu , mFe*chpFe])
    M2 = [-UAr2d+1.1*Rr*Ifd^2*alphaCu 0. 0.; 0. -UAs2Fe+3*Rs*It^2*alphaCu UAs2Fe; 0. UAs2Fe -UAs2Fe-UAFe2a]
    M3 = [0. UAr2d 0.; 0. 0. 0.; 0. 0. UAFe2a]
    #
    N1 = [-mda*chpa mda*chpa+UAr2d 0.; 0. -mda*chpa mda*chpa+UAFe2a; NStw-NSta*
       exp(-NStd) 0. -NStd]
    N2 = [UAr2d 0. 0.; 0. 0. UAFe2a; 0. 0. 0.]
    #
    v = [1.1*Rr*(1-alphaCu*TCu_o)*Ifd^2, 3*Rs*(1-alphaCu*TCu_o)*It^2, QdFes]
    w = [Qdfs, 0., NSta*(1-exp(-NStd))*Twc]
    #
    z = N1\(N2*x + w)
    dxdt = M1\(M2*x+M3*z + v)
    return dxdt
end

# Inputs
u = [3.8,1055.,5360.,212,528,53.9,49.2]
# Initial states
x0 = [28., 28., 28.]
# Time span
tspan = (0., 300*60.)

# findind steady state solution
prob_steady_state = ODEProblem(model2,x0,(0.,Inf),u)
sol_steady_state = solve(prob_steady_state,DynamicSS(Tsit5()))
# steady state state values
xs = sol_steady_state.u[end,1]

┌ Warning: Instability detected. Aborting
└ @ DiffEqBase C:\Users\pande\.julia\packages\DiffEqBase\3gIPB\src\integrator_interface.jl:162


3-element Array{Float64,1}:
 109.49580613517992
  78.99196973704728
  68.75554687333407

In [15]:
# nominal inputs
us = u

using ForwardDiff
# functions in states and inputs
fx(x) = model2(x,us,0)
fu(u) = model2(xs,u,0)
# Jacobians at operating points
A = ForwardDiff.jacobian(fx,xs)
B = ForwardDiff.jacobian(fu,us)

3×7 Array{Float64,2}:
 0.000591328  0.000110906  0.0         …  -2.04558e-5  -2.76863e-5
 0.0          0.0          2.90638e-5      0.0          0.0       
 0.000269202  0.0          0.0            -9.3125e-6   -6.42182e-5

In [16]:
# Calculating time constants
# Eigen value of system matrix, A
using LinearAlgebra
# Eigenvector
R = eigvecs(A)
# Eigenvalues
λ = eigvals(A)
# Time Constants
τ = - @. inv(λ)/60

3-element Array{Float64,1}:
 31.12677552568101  
 68.92052039063147  
  2.0837756429466143

In [17]:
# Controllability and observability
using ControlSystems

# Rank of matrix, A
# System order
rank_of_A = rank(A)
# Controllability matrix
ctrb_C = ctrb(A,B)
# Rank of controlability matrix
rank_of_ctrb_C=rank(ctrb_C)

3

In [18]:
# Observability
C = [1. 1. 0.]
# Observability matrix
obsv_O = obsv(A,C)
rank_of_obsv_O=rank(obsv_O)

3